# Find Candidate Words

Suppose you are working on a puzzle which is one of the many variants of Wordle and are stuck on a word - maybe you know some of the letters but can figure out how to arrange them to find the correct word.

Below is a method you can use to create word hints.

Cesar Mugnatto  
2024-03-03

In [1]:
# Import the necessary libaries
import itertools
import numpy as np
import requests
import json

In [2]:
# We will use this API to obtain a list of possible matching words given the known letters in your word
# Full documentation of the API endpoints and syntax located at https://www.datamuse.com/api/
api_url = "https://api.datamuse.com/words?sp="

As an example, suppose you made 3 guesses which resulted in the following response:

![Guesses](images/guesses.png)

We know that the "r" in position 1 (using a 0-based index) is in the correct spot, and the "a" in position 2 is also correct. We know the letter "t" exists somewhere but not in position 3 which is what we guessed.

Examining the "keyboard" where our guesses have been registered, we see that the letters Q, W, F, H, J, K, Z, X, C, V, B have not been used in any guesses yet, so all are still avaiable.

![Available](images/remaining_letters.png)

We would fill out the parameters `known_letters_loc` and `avail_letters` below with the relevant info.

In [3]:
# The length of the word you want to generate
word_length = 5 # Typical for Wordle

# The letters of the word you already know in the form (letter, position)
# where letter is a single alphabetic character, and position is a value
# between 0 and the length of the word - 1 (0-based indexing)
# or -1 (signifying an unknown position)

known_letters_loc = [
      ('e', 1)
    , ('d', 4)
    , ('i', -1)
    , ('t', -1)
]

# Letters that could still be used - we will add the known letters to these since letters can repeat within words
avail_letters = ['q', 'w', 'f', 'h', 'j', 'k', 'z', 'x', 'c', 'v', 'b']

In [4]:
# First extract the letters for which we know their location
fixed_letters = sorted([l for l in known_letters_loc if l[1] >= 0], key=lambda k: k[1])
fixed_letters

[('e', 1), ('d', 4)]

In [5]:
# For easier processing, let's create  dict with keys = positions, and values = known letter
dict_letters = {x[1]: x[0] for x in fixed_letters}
dict_letters

{1: 'e', 4: 'd'}

In [6]:
# Create our wildcard pattern based on the position of letters whose positions we know, and question marks otherwise
known_pattern = [dict_letters[i] if i in dict_letters.keys() else '?' for i in range(word_length)]
known_pattern

['?', 'e', '?', '?', 'd']

In [7]:
# In order to start replacing widl card characters, we need the positions of unknown letters
unknown_pos = [i for i in range(word_length) if i not in dict_letters.keys()]
unknown_pos

[0, 2, 3]

In [8]:
# Now let's get just the unknwon letters by themselves
unknown_letters = [x[0] for x in sorted([l for l in known_letters_loc if l[1] < 0], key=lambda k: k[1])]
unknown_letters

['i', 't']

In [9]:
# We need to know all of the possible combinations of unknown positions to do character replacement
unknown_combos = list(itertools.combinations(unknown_pos, len(unknown_letters)))
unknown_combos

[(0, 2), (0, 3), (2, 3)]

In [10]:
# Within each possible combinations of unknown positions we also need to permute their positions
perms_unknown_combos = [list(itertools.permutations(x)) for x in unknown_combos]
perms_unknown_combos

[[(0, 2), (2, 0)], [(0, 3), (3, 0)], [(2, 3), (3, 2)]]

In [11]:
# Now finally we can build our patterns that we will feed to the API
known_patterns = []
for c in range(len(perms_unknown_combos)):
    for p in range(len(perms_unknown_combos[c])):
        tmp_pattern = known_pattern.copy()
        #perms_unknown_combos[c][p]:
        #print(list(zip(perms_unknown_combos[c][p], unknown_letters)))
        for x in zip(perms_unknown_combos[c][p], unknown_letters):
            tmp_pattern[x[0]] = x[1]
        known_patterns.append(''.join(tmp_pattern))

known_patterns

['iet?d', 'tei?d', 'ie?td', 'te?id', '?eitd', '?etid']

In [12]:
# We also need just the letters without their position for calling the API
known_letters = [l[0] for l in known_letters_loc]
known_letters

['e', 'd', 'i', 't']

In [13]:
avail_letters.extend(known_letters)
avail_letters

['q', 'w', 'f', 'h', 'j', 'k', 'z', 'x', 'c', 'v', 'b', 'e', 'd', 'i', 't']

In [14]:
# Now, loop through each of the query patterns calling the API
candidate_words = []
for p in known_patterns:
    print(f"Filter: {p}")
    req_url = api_url + p
    resp = requests.get(req_url)
    words = json.loads(resp.text)
    # The API will return many words that do not match our required length so we filter out the non-matching lengths
    good_words = [w['word'] for w in words if len(w['word'])==word_length]
    print(f"Result: {good_words}")
    if len(good_words) > 0:
        # If we still have some words of the correct length remaining, add them to the list
        candidate_words.extend(good_words)

print(f"\nFinal set of words from API of length {word_length}:\n{candidate_words}")

Filter: iet?d
Result: []
Filter: tei?d
Result: ['teiid', 'teind']
Filter: ie?td
Result: []
Filter: te?id
Result: ['tepid', 'teiid']
Filter: ?eitd
Result: []
Filter: ?etid
Result: ['fetid', 'betid', 'netid']

Final set of words from API of length 5:
['teiid', 'teind', 'tepid', 'teiid', 'fetid', 'betid', 'netid']


In [15]:
# Now we can apply filtering on the remaining words where the letter and position are both known
# First convert the letters in all of the words to a numpy array (syntactically easier to filter)
candidate_arr = np.unique(np.array([list(w) for w in candidate_words]), axis=0)
candidate_arr

array([['b', 'e', 't', 'i', 'd'],
       ['f', 'e', 't', 'i', 'd'],
       ['n', 'e', 't', 'i', 'd'],
       ['t', 'e', 'i', 'i', 'd'],
       ['t', 'e', 'i', 'n', 'd'],
       ['t', 'e', 'p', 'i', 'd']], dtype='<U1')

In [16]:
# We filter out any words that contain letters that have been determined not to exist in the word
filtered_candidate_arr = np.array([x for x in candidate_arr.tolist() if len(set(x).difference(set(avail_letters))) == 0])
filtered_candidate_arr

array([['b', 'e', 't', 'i', 'd'],
       ['f', 'e', 't', 'i', 'd'],
       ['t', 'e', 'i', 'i', 'd']], dtype='<U1')

In [17]:
if len(filtered_candidate_arr) < 1:
    print(f'No words found containing the letters {", ".join(avail_letters)}')
else:
    # Convert the array of letters remaining into words remaining and display them
    list_candidate_words = [''.join(l) for l in filtered_candidate_arr]
    for f in fixed_letters:
        filtered_candidate_arr = filtered_candidate_arr[np.argwhere(filtered_candidate_arr[:, f[1]] == f[0]).reshape(1, -1)[0]]
    print('\n'.join(list_candidate_words))# Loop through each of the known letter positions and apply the filter of this letter to its position in the word

betid
fetid
teiid


If we had chosen:

```python
known_letters_loc = [
      ('a', -1)
    , ('i', 1)
    , ('c', 4)
    , ('l', -1)
]
```

and for:

```python
avail_letters = ['q', 'z', 'x', 'j']
```

our result would have been displayed as the only word possible given our criteria:

```text
lilac
```